# Evaluation

In [3]:
import os
import logging
import json
import subprocess
from pathlib import Path
import fnmatch
from random import shuffle
import tempfile

## Setup

### Store Directory

In [4]:
storedir = None  # Set this to persist evaluation results/checkpoints

In [5]:
if storedir is not None:
    checkpoint_storedir = f"{storedir}/checkpoints"
    Path(checkpoint_storedir).mkdir(exist_ok=True)

    data_storedir = f"{storedir}/data"
    Path(data_storedir).mkdir(exist_ok=True)
else:
    checkpoint_storedir = None
    data_storedir = None
    
try:
    job_id = os.environ['PBS_JOBID'].split('.pbs')[0]
except KeyError:
    job_id = 'local'

In [6]:
logging.basicConfig()
logger = logging.getLogger('job')
logger.setLevel(logging.INFO)

### Imports

In [ ]:
logger.info('Importing third-party packages ...')

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

from op_ds.gno.gno import GNOLayer, GNO
from op_ds.gno.kernel import NonlinearKernelTransformWithSkip
from op_ds.utils.fnn import FNN
from volatility_smoothing.utils.gno.train import Trainer
from volatility_smoothing.utils.options_data import CBOEOptionsDataset
from volatility_smoothing.utils.gno.dataset import GNOOptionsDataset
from volatility_smoothing.utils.chunk import chunked


### Device

In [ ]:
logger.info(f"Defining device (torch.cuda.is_available()={torch.cuda.is_available()})")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

logger.info(f'Running using device `{device}`')

if device.type == 'cuda':
    result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
    formatted_result = str(result.stdout).replace('\\n', '\n').replace('\\t', '\t')##

    logger.info(formatted_result)
    logger.info(f'Device count: {torch.cuda.device_count()}')
    logger.info(f'Visible devices count: {os.environ["CUDA_VISIBLE_DEVICES"]}')

## Datasets

In [10]:
data_dir = "../data/cboe"
train_dir = f"{data_dir}/train"
dev_dir = f"{data_dir}/dev"
test_dirs = sorted([f'{data_dir}/' + match for match in fnmatch.filter(os.listdir(data_dir), 'test_*')])


def read_filepaths(dir):
    return [f"{dir}/{filename}" for filename in fnmatch.filter(os.listdir(dir), '*.pt')]



In [ ]:
train_dataset = CBOEOptionsDataset(cache_dir=train_dir)
dev_dataset = CBOEOptionsDataset(cache_dir=dev_dir)
test_datasets = [CBOEOptionsDataset(cache_dir=test_dir) for test_dir in test_dirs]

## Model

In [14]:
in_channels = 1
out_channels = 1
channels = (in_channels, 16, 16, 16, out_channels)
spatial_dim = 2
gno_channels = 16
hidden_channels = 64

gno_layers = []

for i in range(m := (len(channels) - 1)):
    lifting = FNN.from_config((channels[i], hidden_channels, gno_channels), hidden_activation='gelu', batch_norm=False)
    projection = None if i < m - 1 else FNN.from_config((gno_channels, hidden_channels, channels[i+1]), hidden_activation='gelu', batch_norm=False)
    transform = NonlinearKernelTransformWithSkip(in_channels=gno_channels, out_channels=gno_channels, skip_channels=in_channels, spatial_dim=spatial_dim, hidden_channels=(hidden_channels, hidden_channels), hidden_activation='gelu', batch_norm=False)

    if i == 0:
        local_linear = False
    else:
        local_linear = True
        
    activation = torch.nn.GELU() if i < m - 1 else torch.nn.Softplus(beta=0.5)
        
    gno_layer = GNOLayer(gno_channels, transform=transform, local_linear=local_linear, local_bias=True,
                         activation=activation, lifting=lifting, projection=projection)
    gno_layers.append(gno_layer)
    
gno = GNO(*gno_layers, in_channels=in_channels).to(device)

In [15]:
def load_checkpoint(model, optimizer, path):
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    logger.info(f"Loaded checkpoint from {path}")
    return model, optimizer

In [16]:
path = "../train/store/9448705/checkpoints/checkpoint_final.pt"

In [ ]:
optimizer = torch.optim.AdamW(gno.parameters())
gno, optimizer = load_checkpoint(gno, optimizer, path)

## Evaluation/Finetuning Hyperparameters

In [18]:
lr = 1e-4
weight_decay = 1e-5
epochs = 10  # Finetune epochs, set to 0 to skip and just evaluate
batch_size = 64  # Finetune batch size, will be augmented by same amount of training data

# mesh sizes on which to evaluate arbitrage metrics
step_r = 0.02
step_z = 0.01

In [ ]:
trainer = Trainer()  # Use default parameters
gno_train_dataset = GNOOptionsDataset(train_dataset, subsample=True)
gno_dev_dataset = GNOOptionsDataset(dev_dataset, subsample=False)
gno_test_datasets = [GNOOptionsDataset(test_dataset, subsample=False) for test_dataset in test_datasets]
optimizer = torch.optim.AdamW(gno.parameters(), lr=lr, weight_decay=weight_decay)

## Evaluation

In [15]:
num_workers = 0

In [ ]:
df_val, df_rel, df_fit = trainer.evaluate(gno, gno_dev_dataset, device=device,
                                  num_workers=num_workers, storedir=storedir, logger=logger,
                                  step_r=step_r, step_z=step_z)

In [ ]:
torch.cuda.empty_cache()

logger.info(50 * "=")
logger.info(f"Evaluation start (Retraining epochs: {epochs}).")
logger.info(50 * "=")



with tempfile.TemporaryDirectory() as tmpdir:  # Create empty temporary directory (to init with empty list)
    gno_finetune_dataset = GNOOptionsDataset(CBOEOptionsDataset(cache_dir=tmpdir), subsample=True)

model = gno.to(device).train()
try:
    for k, gno_test_dataset in enumerate(gno_test_datasets):

        logger.info(f"Evaluating model")
        model.eval()
        df_val, df_rel, df_fit  = trainer.evaluate(model, gno_test_dataset, device=device,
                                num_workers=num_workers, storedir=storedir, logger=logger,
                                step_r=step_r, step_z=step_z)
    
        model.train()

        gno_finetune_dataset.options_dataset._data += gno_test_dataset.options_dataset._data
        idx_list = list(range(len(gno_finetune_dataset)))
        
        for epoch in range(epochs):

            logger.info(f'Finetune step {k}. Epoch {epoch}/{epochs}')
            logger.info(f"Loss weights: {trainer.error_weights}")
        
            shuffle(idx_list)
        
            t_dataloader = DataLoader(gno_train_dataset, batch_size=1, collate_fn=trainer.collate_fn, shuffle=True, num_workers=num_workers, pin_memory=False)
            f_dataloader = DataLoader(gno_finetune_dataset, batch_size=1, collate_fn=trainer.collate_fn, shuffle=True, num_workers=num_workers, pin_memory=False)

            t_its = iter(t_dataloader)
            f_its = iter(f_dataloader)
            
            for count, batch_idx in zip(range(len(idx_list)), (iterations := tqdm(chunked(idx_list, batch_size)))):
            
                bs = 2 * len(batch_idx)
            
                optimizer.zero_grad()
            
                batch_loss = 0

                loss_infos = []
                loss_str = []
            
                for i in batch_idx:
                
                    # Base data
                    data, input, aux = next(t_its)
                    data = data.to(device)
                    input = {key: val.to(device) for key, val in input.items()}
                    aux['grids'] = [grid.to(device) for grid in aux['grids']]
                
                    output = model(**input)
                    sample_loss, sample_loss_info = trainer.loss(data, output, aux)                
                    sample_loss = sample_loss / bs
                    sample_loss.backward()
                
                    batch_loss =  batch_loss + sample_loss
                    loss_infos.append(sample_loss_info)
                
                    # Finetune data
                    data, input, aux = next(f_its)
                    data = data.to(device)
                    input = {key: val.to(device) for key, val in input.items()}
                    aux['grids'] = [grid.to(device) for grid in aux['grids'] if grid is not None]

                    output = model(**input)
                    sample_loss, sample_loss_info = trainer.loss(data, output, aux)                
                    sample_loss = sample_loss / bs
                    sample_loss.backward()

                    batch_loss = batch_loss + sample_loss
                    loss_infos.append(sample_loss_info)

                loss_details = {k: [loss_info[k] for loss_info in loss_infos] for k in loss_infos[0]}
                loss_str.append(f"mape: {sum(loss_details['mape']) / bs :> 10.3g}")
                loss_str.append(f"wmape: {sum(loss_details['wmape']) / bs :> 10.3g}")
                loss_str.append(f"fit pen: {sum(loss_details['fit']) / bs :> 10.3g}")
                loss_str.append(f"cal pen: {sum(loss_details['cal']) / bs :> 10.3g}")
                loss_str.append(f"but pen: {sum(loss_details['but']) / bs :> 10.3g}")
                loss_str.append(f"reg_z pen: {sum(loss_details['reg_z']) / bs :> 10.3g}")
                loss_str.append(f"reg_r pen: {sum(loss_details['reg_r']) / bs :> 10.3g}")
                                        
                loss_s = f"loss: {batch_loss: .8f} ({', '.join(loss_str)})"
                iterations.set_description(loss_s)

                if (iterations.n % 10 == 0) and (storedir is not None):
                    logger.info(f"{k}-{epoch}-{len(iterations)}-{iterations.n} -- {loss_s}")                                
                
                optimizer.step()

            df_val, df_rel, df_fit = trainer.evaluate(model, gno_dev_dataset, device=device, num_workers=num_workers)
            logger.info(f"Epoch {epoch} Dev: {df_val.describe()}")
            df_val.to_csv(f"{checkpoint_storedir}/val_{k}-{epoch}.csv")
            model.train()

            if checkpoint_storedir is not None:
                checkpoint = {
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                }
            torch.save(checkpoint, f"{checkpoint_storedir}/checkpoint_{k}-{epoch}.pt")

except KeyboardInterrupt:
    try:
        batch_loss.backward()
    except:
        pass
    logging.info("Training aborted")
else:
    logging.info("Training complete")
finally:
    if checkpoint_storedir is not None:
        checkpoint = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        torch.save(model, f"{checkpoint_storedir}/checkpoint_final.pt")
    model.eval()